In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load data path
data_path = Path("/Users/shriyankatiki/Documents/datasys/safestreets/data/raw/US_Accidents_March23.csv")
print(f"Loading data from: {data_path}")
print(f"File exists: {data_path.exists()}")

# Read first 100k rows to explore
print("\n📥 Reading first 100,000 rows...")
df = pd.read_csv(data_path, nrows=100000)

print(f"\n📊 Dataset Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

In [ ]:
# Column names
print("📋 Column Names:")
print(df.columns.tolist())

print("\n📝 Data Types:")
print(df.dtypes)

print("\n🔍 First 5 rows:")
df.head()

In [ ]:
# Check for missing values
print("❓ Missing Values Analysis:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
}).sort_values('Missing_Count', ascending=False)

# Show only columns with missing values
print(missing_df[missing_df['Missing_Count'] > 0])

In [ ]:
# Severity distribution
print("⚠️ Severity Distribution:")
severity_counts = df['Severity'].value_counts().sort_index()
print(severity_counts)
print(f"\nPercentages:")
print((severity_counts / len(df) * 100).round(2))

# Date range
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
print(f"\n📅 Date Range:")
print(f"Earliest: {df['Start_Time'].min()}")
print(f"Latest: {df['Start_Time'].max()}")

# Top states
print("\n🗺️ Top 10 States by Accident Count:")
state_counts = df['State'].value_counts().head(10)
print(state_counts)

In [ ]:
# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Severity distribution plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Severity bar chart
df['Severity'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Accident Severity Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Severity Level')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Top 10 states
df['State'].value_counts().head(10).plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Top 10 States by Accident Count', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Number of Accidents')
axes[1].set_ylabel('State')

plt.tight_layout()
plt.show()

In [ ]:
# Extract time components
df['Hour'] = df['Start_Time'].dt.hour
df['DayOfWeek'] = df['Start_Time'].dt.dayofweek
df['Month'] = df['Start_Time'].dt.month
df['Year'] = df['Start_Time'].dt.year

# Accidents by hour
print("🕐 Accidents by Hour of Day:")
hourly = df['Hour'].value_counts().sort_index()
print(hourly)

# Plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hourly distribution
hourly.plot(kind='bar', ax=axes[0, 0], color='teal')
axes[0, 0].set_title('Accidents by Hour of Day', fontweight='bold')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Count')

# Day of week (0=Monday, 6=Sunday)
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow_counts = df['DayOfWeek'].value_counts().sort_index()
dow_counts.index = day_names
dow_counts.plot(kind='bar', ax=axes[0, 1], color='salmon')
axes[0, 1].set_title('Accidents by Day of Week', fontweight='bold')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# Monthly distribution
df['Month'].value_counts().sort_index().plot(kind='bar', ax=axes[1, 0], color='purple')
axes[1, 0].set_title('Accidents by Month', fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Count')

# Yearly trend
df['Year'].value_counts().sort_index().plot(kind='line', ax=axes[1, 1], marker='o', color='darkblue')
axes[1, 1].set_title('Accidents by Year', fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Count')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Weather analysis
print("🌦️ Top 10 Weather Conditions:")
weather = df['Weather_Condition'].value_counts().head(10)
print(weather)

# Plot
plt.figure(figsize=(12, 6))
weather.plot(kind='barh', color='skyblue')
plt.title('Top 10 Weather Conditions During Accidents', fontsize=14, fontweight='bold')
plt.xlabel('Number of Accidents')
plt.ylabel('Weather Condition')
plt.tight_layout()
plt.show()

In [ ]:
# Numerical columns summary
print("📊 Summary Statistics for Key Numerical Columns:")
numerical_cols = ['Severity', 'Distance(mi)', 'Temperature(F)', 
                  'Visibility(mi)', 'Precipitation(in)']

print(df[numerical_cols].describe())

In [ ]:
print("=" * 60)
print("🎯 KEY INSIGHTS FROM EXPLORATION")
print("=" * 60)

print(f"\n1. DATASET SIZE:")
print(f"   - Total rows in sample: {len(df):,}")
print(f"   - Total columns: {df.shape[1]}")
print(f"   - Estimated full dataset: ~7.7 million records")

print(f"\n2. SEVERITY:")
severity_pct = (df['Severity'].value_counts(normalize=True) * 100).sort_index()
for sev, pct in severity_pct.items():
    print(f"   - Severity {sev}: {pct:.1f}%")

print(f"\n3. GEOGRAPHIC COVERAGE:")
print(f"   - Number of states: {df['State'].nunique()}")
print(f"   - Top state: {df['State'].value_counts().index[0]} ({df['State'].value_counts().iloc[0]:,} accidents)")

print(f"\n4. TEMPORAL PATTERNS:")
print(f"   - Date range: {df['Start_Time'].min().date()} to {df['Start_Time'].max().date()}")
print(f"   - Peak hour: {df['Hour'].mode()[0]}:00")
print(f"   - Peak day: {day_names[df['DayOfWeek'].mode()[0]]}")

print(f"\n5. DATA QUALITY:")
missing_cols = missing_df[missing_df['Missing_Count'] > 0].shape[0]
print(f"   - Columns with missing data: {missing_cols}")
print(f"   - Most missing: {missing_df.iloc[0]['Column']} ({missing_df.iloc[0]['Missing_Percentage']:.1f}%)")

print(f"\n6. NEXT STEPS:")
print("   - Convert to Parquet format for efficiency")
print("   - Handle missing values in critical columns")
print("   - Create geographic aggregations (city/state level)")
print("   - Build time-based features")
print("   - Identify high-risk road segments")

print("\n" + "=" * 60)